문제 1: 채널별 고객 획득 비용(CAC) 및 생애 가치(LTV) 분석
목표: 마케팅 채널의 진정한 ROI를 평가하기 위해 CAC와 LTV를 계산하고 최적 투자 전략을 제안하세요.
1. 데이터 로드 및 전처리
2. 채널별 CAC 계산
    - 2024년 데이터만 사용
    - 각 채널별 총 마케팅 비용 집계
    - 각 채널별 신규 고객 수 집계 (2024년 첫 거래 기준)
    - CAC = 총 마케팅 비용 / 신규 고객 수
    - organic과 referral 채널은 CAC = 0으로 처리
3. 고객별 LTV 계산
4. 채널별 LTV 및 ROI 분석
    - 채널별 평균 LTV 계산
    - ROI = LTV / CAC 계산 (organic, referral은 무한대 처리)
    - payback period = CAC / (월평균 구매금액) 계산
5. 시각화 및 전략 제안 (15분)
    - CAC vs LTV 산점도 차트
    - 채널별 ROI 막대 차트
    - 마케팅 예산 재배분 제안 (현재 vs 최적 배분)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
plt.rcParams['font.family'] = 'Pretendard'
plt.rcParams['axes.unicode_minus'] = False

# 2. pd.read_csv()로 읽어들여서 DataFrame 생성
cust_satis = pd.read_csv('customer_satisfaction.csv',        # 혹은 '폴더/파일명.csv', '/절대경로/파일.csv'
                 encoding='utf-8',  # 한글 포함 파일은 'cp949' 등으로 바꿔볼 수 있다
                 sep=',',          # 구분자가 ,가 아니면 '\t'나 ';' 등으로 지정
                )
cust_trans = pd.read_csv('customer_transactions.csv',        # 혹은 '폴더/파일명.csv', '/절대경로/파일.csv'
                 encoding='utf-8',  # 한글 포함 파일은 'cp949' 등으로 바꿔볼 수 있다
                 sep=',',          # 구분자가 ,가 아니면 '\t'나 ';' 등으로 지정
                )
maerket_perf = pd.read_csv('marketing_performance.csv',        # 혹은 '폴더/파일명.csv', '/절대경로/파일.csv'
                 encoding='utf-8',  # 한글 포함 파일은 'cp949' 등으로 바꿔볼 수 있다
                 sep=',',          # 구분자가 ,가 아니면 '\t'나 ';' 등으로 지정
                )
                

# 결측치 확인
cust_satis.isna().sum()
cust_trans.isna().sum()
maerket_perf.isna().sum()

cust_satis.describe(), cust_trans.describe(), maerket_perf.describe()
customers = cust_satis.merge(cust_trans, on='customer_id')
customers

# 2. 채널별 CAC 계산
#     - 2024년 데이터만 사용
#     - 각 채널별 총 마케팅 비용 집계
#     - 각 채널별 신규 고객 수 집계 (2024년 첫 거래 기준)
#     - CAC = 총 마케팅 비용 / 신규 고객 수
#     - organic과 referral 채널은 CAC = 0으로 처리

mp_2024 = maerket_perf[maerket_perf['month'] >= '2024-01-01']


mp_2024_summary = mp_2024.groupby('channel').agg({
    'marketing_spend': ['sum', 'mean', 'count'],
    'new_customers': 'nunique'
}).reset_index()
# sum().reset_index()
# print(mp_2024_summary)

mp_2024_summary['CAC'] = mp_2024_summary['marketing_spend']['sum'] / mp_2024_summary['new_customers']['nunique']
# print(mp_2024_summary)

mp_2024_summary.loc[
    mp_2024_summary['channel'].isin(['organic', 'referral']),
    'CAC'
] = 0

print(mp_2024_summary)

        channel marketing_spend                    new_customers           CAC
                            sum         mean count       nunique              
0         email    17763.268813  1480.272401    12             6   2960.544802
1       organic        0.000000     0.000000    12             9      0.000000
2   paid_search    97849.029394  8154.085783    12             8  12231.128674
3      referral    11867.432377   988.952698    12             9      0.000000
4  social_media    59628.676756  4969.056396    12             7   8518.382394


In [19]:
customers['transaction_date'] = pd.to_datetime(customers['transaction_date'])
customers['month'] = customers['transaction_date'].dt.to_period('M')
customers
# .groupby('transaction_date').size()

,customer_id,contact_date,contact_reason,satisfaction_score,resolution_time_hours,repeat_contact,registration_date,acquisition_channel,customer_segment,transaction_date,order_value,product_category,month
0,495,2023-06-16,technical,4.0,11.4,False,2023-05-30,paid_search,standard,2023-06-01,67.87,Books,2023-06
1,495,2023-06-16,technical,4.0,11.4,False,2023-05-30,paid_search,standard,2024-05-29,59.04,Electronics,2024-05
2,495,2023-06-16,technical,4.0,11.4,False,2023-05-30,paid_search,standard,2024-04-10,84.73,Home,2024-04
3,495,2023-06-16,technical,4.0,11.4,False,2023-05-30,paid_search,standard,2023-08-31,58.16,Books,2023-08
4,495,2024-03-21,shipping,1.8,8.1,True,2023-05-30,paid_search,standard,2023-06-01,67.87,Books,2023-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5244,948,2023-07-15,billing,2.8,5.5,False,2023-10-31,social_media,standard,2024-03-26,54.64,Fashion,2024-03
5245,948,2023-07-15,billing,2.8,5.5,False,2023-10-31,social_media,standard,2024-08-11,87.91,Fashion,2024-08
5246,948,2023-07-15,billing,2.8,5.5,False,2023-10-31,social_media,standard,2024-03-03,76.13,Home,2024-03
5247,948,2023-07-15,billing,2.8,5.5,False,2023-10-31,social_media,standard,2024-01-31,75.80,Home,2024-01


In [10]:
# 3. 고객별 LTV 계산
"""
CLV = (평균 구매 금액) * (평균 구매 빈도) * (평균 고객 생애 기간)
"""

# 평균 구매 금액
avg_cust_buy = customers.groupby('customer_id')['order_value'].mean()
# print(avg_cust_buy)

# 평균 연간 구매 빈도
customers['contact_date'] = pd.to_datetime(customers['contact_date'])
avg_frequency = customers.groupby('customer_id').apply(lambda x: len(x) / ((x['contact_date'].max() - x['contact_date'].min()).days + 1) * 365)
print(avg_frequency)

# 평균 고객 생애 기간.
lifespan_days = customers.groupby('customer_id').apply(
        lambda x: (x['contact_date'].max() - x['contact_date'].min()).days + 1
    )
# print(lifespan_days)

# customers['clv']

clv = avg_cust_buy.values * avg_frequency.values * lifespan_days.values

clv
clv_data = pd.DataFrame({
    'customer_id': avg_cust_buy.index,
    'avg_order_value': avg_cust_buy.values,
    'avg_frequency': avg_frequency.values,
    'lifespan_days': lifespan_days.values,
    'clv': clv
})

clv_data

 # 확인용
# customers['contact_date'] = pd.to_datetime(customers['contact_date'])
# last_buy = customers.groupby('customer_id')['contact_date'].max()
# first_buy = customers.groupby('customer_id')['contact_date'].min()
# cust_period = last_buy - first_buy
# print(cust_period)


customer_id
1        32.686567
2        97.333333
3        19.796610
5        12.248322
7      1825.000000
          ...     
992    1095.000000
993      16.102941
996       4.949153
997     730.000000
998      32.589286
Length: 400, dtype: float64


/var/folders/tx/t7px7cn503bgk6zlg3m0b4740000gn/T/ipykernel_74719/1043024861.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_frequency = customers.groupby('customer_id').apply(lambda x: len(x) / ((x['contact_date'].max() - x['contact_date'].min()).days + 1) * 365)
/var/folders/tx/t7px7cn503bgk6zlg3m0b4740000gn/T/ipykernel_74719/1043024861.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lifespan_days = cu

,customer_id,avg_order_value,avg_frequency,lifespan_days,clv
0,1,117.250000,32.686567,268,1027110.00
1,2,110.125000,97.333333,45,482347.50
2,3,68.588750,19.796610,295,400558.30
3,5,75.770000,12.248322,298,276560.50
4,7,72.350000,1825.000000,1,132038.75
...,...,...,...,...,...
395,992,74.596667,1095.000000,1,81683.35
396,993,61.795000,16.102941,136,135331.05
397,996,44.130000,4.949153,295,64429.80
398,997,44.415000,730.000000,1,32422.95


In [12]:
maerket_perf

,month,channel,marketing_spend,new_customers,website_visits,conversions
0,2023-01-01,organic,0.000000,28,511,28
1,2023-01-01,paid_search,11986.882840,19,995,19
2,2023-01-01,social_media,4235.453095,17,1048,17
3,2023-01-01,email,1693.773605,17,1037,17
4,2023-01-01,referral,887.489645,10,958,10
...,...,...,...,...,...,...
115,2024-12-01,organic,0.000000,27,505,27
116,2024-12-01,paid_search,6149.273297,20,999,20
117,2024-12-01,social_media,4668.252208,14,1028,14
118,2024-12-01,email,1496.013316,9,1013,9


In [13]:
# 4. 채널별 LTV 및 ROI 분석
#     - 채널별 평균 LTV 계산
#     - ROI = LTV / CAC 계산 (organic, referral은 무한대 처리)
#     - payback period = CAC / (월평균 구매금액) 계산

maerket_perf.groupby('channel').agg({
    'marketing_spend': 'mean',
    
})

# 채널 평균 매출

# 채널 평균빈도
# 채널 평균 생애 기간.

,marketing_spend
channel,
email,1518.458591
organic,0.000000
paid_search,8126.860601
referral,997.667423
social_media,4903.568263


문제 2: 고객 코호트 분석 및 유지율 개선 전략
목표: "월별 가입 코호트"의 유지율과 매출 기여도를 분석하여 고객 유지 전략을 수립하세요.
1. 코호트 그룹 생성
2. 월별 유지율 계산
    - 각 거래의 코호트 대비 경과 개월 수 계산
    - 코호트별 Month 0, 1, 2, ... 12에서의 활성 고객 수
    - 유지율 테이블 생성 (코호트 × 경과월)
    - 평균 유지율 곡선 계산
3. 코호트별 매출 기여도 분석
    - 각 코호트의 월별 누적 매출 계산
    - 코호트별 고객 1인당 누적 매출 (ARPU) 계산
    - 6개월 후, 12개월 후 예상 LTV 추정
4. 유지율 히트맵 시각화
5. 개선 전략 수립
    - 유지율이 급격히 감소하는 시점 식별
    - 고성과 코호트 vs 저성과 코호트 특성 비교
    - 유지율 개선을 위한 액션 아이템 3가지 제안

In [ ]:
maerket_perf

,month,channel,marketing_spend,new_customers,website_visits,conversions
0,2023-01-01,organic,0.000000,28,511,28
1,2023-01-01,paid_search,11986.882840,19,995,19
2,2023-01-01,social_media,4235.453095,17,1048,17
3,2023-01-01,email,1693.773605,17,1037,17
4,2023-01-01,referral,887.489645,10,958,10
...,...,...,...,...,...,...
115,2024-12-01,organic,0.000000,27,505,27
116,2024-12-01,paid_search,6149.273297,20,999,20
117,2024-12-01,social_media,4668.252208,14,1028,14
118,2024-12-01,email,1496.013316,9,1013,9
